In [30]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error

import xgboost as xgb
from scripts.benchmark import DATA_DIR, DATASETS, config_data,neg_log_likelihood

In [ ]:
X_train, X_test, y_train, y_test = config_data("airfoil")
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(1202, 5)
(301, 5)
(1202,)
(301,)
(1, 301)


In [32]:
reg = xgb.XGBRegressor(
    tree_method="hist",
    n_estimators=2000,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=0,
)

reg.fit(
    X_train, y_train,
    verbose=False,
)

pred = reg.predict(X_test)
rmse = root_mean_squared_error(y_test, pred)
print("rmse", root_mean_squared_error(y_test, pred))
print(f"nll { neg_log_likelihood(y_test, pred, root_mean_squared_error(y_test, pred)).mean()}")

rmse 0.1945188045501709
nll -0.21828795167825576


In [33]:
from sklearn.ensemble import RandomForestRegressor
mlp= RandomForestRegressor(n_estimators=300)
mlp.fit(X_train,y_train)
mlp_pred= mlp.predict(X_test)
print("rmse", root_mean_squared_error(y_test, mlp_pred))
print(f"nll { neg_log_likelihood(y_test, pred, root_mean_squared_error(y_test, mlp_pred)).mean()}")

rmse 0.26582225259715986
nll -0.13825059665472855


In [ ]:
from sklearn.linear_model import LinearRegression
mlp = LinearRegression(fit_intercept=False)
mlp.fit(X_train,y_train)
mlp_pred = mlp.predict(X_test)
print("rmse", root_mean_squared_error(y_test, mlp_pred))
print(f"nll { neg_log_likelihood(y_test, pred, root_mean_squared_error(y_test, mlp_pred)).mean()}")

rmse 0.6867814064025879
nll 0.5833096267562733


In [42]:
from catboost import CatBoostRegressor
cat=CatBoostRegressor(depth=6, iterations=1000, learning_rate=0.05)
cat.fit(X_train,y_train)
cat_pred = cat.predict(X_test)
print("rmse", root_mean_squared_error(y_test, cat_pred))
print(f"nll { neg_log_likelihood(y_test, pred, root_mean_squared_error(y_test, cat_pred)).mean()}")

0:	learn: 0.9767437	total: 1.63ms	remaining: 1.63s
1:	learn: 0.9549973	total: 2.9ms	remaining: 1.45s
2:	learn: 0.9310037	total: 3.72ms	remaining: 1.24s
3:	learn: 0.9128200	total: 4.54ms	remaining: 1.13s
4:	learn: 0.8899329	total: 5.26ms	remaining: 1.05s
5:	learn: 0.8720766	total: 5.94ms	remaining: 985ms
6:	learn: 0.8562042	total: 6.61ms	remaining: 938ms
7:	learn: 0.8379556	total: 7.52ms	remaining: 932ms
8:	learn: 0.8199306	total: 8.87ms	remaining: 976ms
9:	learn: 0.8045970	total: 9.59ms	remaining: 949ms
10:	learn: 0.7870368	total: 10.2ms	remaining: 918ms
11:	learn: 0.7731777	total: 10.8ms	remaining: 892ms
12:	learn: 0.7562408	total: 11.5ms	remaining: 870ms
13:	learn: 0.7427784	total: 12.1ms	remaining: 849ms
14:	learn: 0.7301997	total: 12.7ms	remaining: 832ms
15:	learn: 0.7173943	total: 13.3ms	remaining: 816ms
16:	learn: 0.7036907	total: 13.9ms	remaining: 804ms
17:	learn: 0.6896145	total: 14.5ms	remaining: 791ms
18:	learn: 0.6761310	total: 15.1ms	remaining: 781ms
19:	learn: 0.6655755	to

In [48]:
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor(hidden_layer_sizes=[4, 16, 32, 63, 128, 256, 512, 256, 64, 8], solver="lbfgs", max_iter=5000)
mlp.fit(X_train, y_train)
mlp_pred = mlp.predict(X_test)
print("rmse", root_mean_squared_error(y_test, mlp_pred))
print(
    f"nll { neg_log_likelihood(y_test, pred, root_mean_squared_error(y_test, mlp_pred)).mean()}"
)

rmse 0.31147069100326463
nll -0.05250065171112102


c:\Users\vyron\.conda\envs\bde\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:606: ConvergenceWarning: lbfgs failed to converge after 5000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=5000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
